In [1]:
import requests
import json
import pandas as pd
import numpy as np

In [2]:
BASE_URL = 'https://the-value-crew.github.io/nepse-api/'

In [3]:
def get_response_data(url):
    response = requests.get(url)
    data = response.json()
    return data
    

In [4]:
company_url = 'data/companies.json'
main_url = BASE_URL+company_url
d = get_response_data(main_url)
companies_code = list(d.keys())


In [5]:
company_list = {}
for code in companies_code:
    url = BASE_URL+'data/company/{c}'.format(c=code)+'.json'
    print(url)
    try:
        data = get_response_data(url)
        df = pd.DataFrame(data).T
        df.index.name="date" # naming the index column
        df=pd.concat([df.drop(["price"], axis=1), 
                      df["price"].apply(pd.Series)], axis=1) # this takes out the items i.e. min, max etc. from the 
                                                            # dictionary and makes them a separate column
        df.rename(columns={"close":code}, inplace=True)
        company_list[code] = df
        
    except Exception:
        print('something went wrong') 
    

https://the-value-crew.github.io/nepse-api/data/company/CBBL.json
https://the-value-crew.github.io/nepse-api/data/company/NIL.json
https://the-value-crew.github.io/nepse-api/data/company/GMFIL.json
https://the-value-crew.github.io/nepse-api/data/company/SIL.json
https://the-value-crew.github.io/nepse-api/data/company/PROFL.json
https://the-value-crew.github.io/nepse-api/data/company/AHPC.json
https://the-value-crew.github.io/nepse-api/data/company/ALICL.json
https://the-value-crew.github.io/nepse-api/data/company/KSBBL.json
https://the-value-crew.github.io/nepse-api/data/company/SHINE.json
https://the-value-crew.github.io/nepse-api/data/company/GRDBL.json
https://the-value-crew.github.io/nepse-api/data/company/NMBHF1.json
https://the-value-crew.github.io/nepse-api/data/company/AKJCL.json
https://the-value-crew.github.io/nepse-api/data/company/ACLBSL.json
https://the-value-crew.github.io/nepse-api/data/company/GHL.json
https://the-value-crew.github.io/nepse-api/data/company/ALBSL.json
h

https://the-value-crew.github.io/nepse-api/data/company/KRBL.json
https://the-value-crew.github.io/nepse-api/data/company/MPFL.json
https://the-value-crew.github.io/nepse-api/data/company/SHPC.json
https://the-value-crew.github.io/nepse-api/data/company/CLBSL.json
https://the-value-crew.github.io/nepse-api/data/company/FOWAD.json
https://the-value-crew.github.io/nepse-api/data/company/NADEP.json
https://the-value-crew.github.io/nepse-api/data/company/KPCL.json
https://the-value-crew.github.io/nepse-api/data/company/KBLD86.json
https://the-value-crew.github.io/nepse-api/data/company/NICLBSL.json
https://the-value-crew.github.io/nepse-api/data/company/SDLBSL.json
https://the-value-crew.github.io/nepse-api/data/company/SSHL.json
https://the-value-crew.github.io/nepse-api/data/company/CGH.json
https://the-value-crew.github.io/nepse-api/data/company/RURU.json
https://the-value-crew.github.io/nepse-api/data/company/RULB.json
https://the-value-crew.github.io/nepse-api/data/company/SPHL.json
h

https://the-value-crew.github.io/nepse-api/data/company/NIBSF2.json
https://the-value-crew.github.io/nepse-api/data/company/JBLB.json
https://the-value-crew.github.io/nepse-api/data/company/MLBS.json
https://the-value-crew.github.io/nepse-api/data/company/SAHAS.json
https://the-value-crew.github.io/nepse-api/data/company/DORDI.json
https://the-value-crew.github.io/nepse-api/data/company/PBD88.json
https://the-value-crew.github.io/nepse-api/data/company/PPL.json
https://the-value-crew.github.io/nepse-api/data/company/SIKLES.json


In [7]:
tickers=list(company_list.values()) # converting the dict to list
nan_value=0

In [9]:
final_df = pd.concat(tickers, join='outer', axis=1).fillna(nan_value) # conactenating all the dfs into 1 single df

final_df.drop(['numTrans', 'tradedShares',"amount", "max", 
               "min", "prevClose", "diff", "open"], axis=1, inplace=True) # dropping all the other columns except the "close"

final_df.index= pd.to_datetime(final_df.index) # converting the type of index column to a datetime object

final_df=final_df.sort_index() # sorting the index column in an ascending manner

In [22]:
# saving this final_df as csv file

final_df.to_csv("Analysis_file.csv")

#### Creating a sql Lite DataBase and storing the above final_df into the DB

In [23]:
import sqlalchemy

In [45]:
# creating the db engine

def create_engine(name):
    engine=sqlalchemy.create_engine("sqlite:///" + name)
    return engine

In [46]:
nepse_engine=create_engine("Nepse")

In [47]:
# Transferring the data to the sql db

def tosql(list_ofdfs, symbols, engine):
    for df, symbol in zip(list_ofdfs, symbols):
        df=df.to_sql(symbol, engine, index=True)
    
    print("success")
        

In [48]:
tosql(tickers, companies_code, nepse_engine)

success


In [53]:
# So now, if I wanna call any ticker from the db that we just created:

pd.read_sql("NABIL", nepse_engine)

,date,numTrans,tradedShares,amount,max,min,NABIL,prevClose,diff,open
0,2022-02-20,1188,61489,65233093.3,1075.0,1045.0,1062.0,1079.0,-17.0,NaN
1,2022-02-21,804,39297,41953268,1073.0,1063.0,1067.0,1062.0,5.0,NaN
2,2022-02-22,719,41743,44701700.4,1080.0,1069.0,1070.5,1067.0,3.5,NaN
3,2022-02-23,612,29641,31684313.2,1085.0,1065.0,1067.5,1070.5,-3.0,NaN
4,2022-02-24,1372,72385,76104471.7,1067.0,1048.0,1051.0,1067.5,-16.5,NaN
...,...,...,...,...,...,...,...,...,...,...
2880,2022-12-22,724,66052,53166698,811.0,800.1,810.0,807.0,3.0,809.0
2881,2022-12-26,1101,110057,91121150.2,840.0,811.0,838.0,810.0,28.0,811.0
2882,2022-12-27,1497,133855,113229439,860.0,834.0,842.0,838.0,4.0,847.0
2883,2022-12-28,1074,105429,88591274.1,847.0,826.1,834.9,842.0,-7.1,845.0


In [54]:
# One can also use sql queries to give some conditions

pd.read_sql("SELECT * FROM NABIL WHERE NABIL>prevClose", nepse_engine)

,date,numTrans,tradedShares,amount,max,min,NABIL,prevClose,diff,open
0,2022-02-21,804,39297,41953268,1073.0,1063.0,1067.0,1062.0,5.0,NaN
1,2022-02-22,719,41743,44701700.4,1080.0,1069.0,1070.5,1067.0,3.5,NaN
2,2010-04-22,17,1197,2156607,1831.0,1775.0,1831.0,1810.0,21.0,NaN
3,2010-04-29,12,622,1132280,1838.0,1704.0,1838.0,1671.0,167.0,NaN
4,2010-05-09,1,60,110400,1840.0,1840.0,1840.0,1838.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1260,2022-12-21,654,48877,39508915.7,816.0,800.0,807.0,800.0,7.0,816.0
1261,2022-12-22,724,66052,53166698,811.0,800.1,810.0,807.0,3.0,809.0
1262,2022-12-26,1101,110057,91121150.2,840.0,811.0,838.0,810.0,28.0,811.0
1263,2022-12-27,1497,133855,113229439,860.0,834.0,842.0,838.0,4.0,847.0
